In [10]:
import sqlite3
import pandas as pd

bias_df = pd.read_csv("allsides-ranking.csv")

db_path = "data/news_articles.db"   # update if needed
conn = sqlite3.connect(db_path)

In [16]:
conn.execute("ALTER TABLE article_urls ADD COLUMN bias TEXT;")

In [ ]:
cur = conn.cursor()

for _, row in bias_df.iterrows():
    cur.execute("""
        UPDATE article_urls
        SET bias = ?
        WHERE lower(outlet_name) = lower(?)
    """, (row["bias"], row["outlet"]))

conn.commit()

In [18]:
# article_urls
print("article_urls")
display(pd.read_sql("PRAGMA table_info(article_urls);", conn))

# article_contents
print("article_contents")
display(pd.read_sql("PRAGMA table_info(article_contents);", conn))

=== article_urls ===


,cid,name,type,notnull,dflt_value,pk
0,0,uuid,TEXT,0,None,0
1,1,url,TEXT,0,None,0
2,2,outlet_name,TEXT,0,None,0
3,3,bias,TEXT,0,None,0


=== article_contents ===


,cid,name,type,notnull,dflt_value,pk
0,0,uuid,TEXT,0,None,0
1,1,date,TEXT,0,None,0
2,2,content,TEXT,0,None,0
3,3,content_preprocessed,TEXT,0,None,0
4,4,language,TEXT,0,None,0


In [19]:
# filter to number of articles per year to find viable outlets

query_long = """
SELECT
  u.outlet_name,
  COALESCE(u.bias, 'Unknown') AS bias,
  SUBSTR(c.date,1,4) AS year,
  COUNT(*) AS n
FROM article_urls u
JOIN article_contents c
  ON u.uuid = c.uuid
WHERE c.date IS NOT NULL
  AND c.date >= '2015-01-01'
  AND c.date <  '2022-01-01'
GROUP BY u.outlet_name, COALESCE(u.bias,'Unknown'), SUBSTR(c.date,1,4);
"""
df_long = pd.read_sql(query_long, conn)

# Pivot to years as columns
df_pivot = (df_long
            .pivot_table(index=['outlet_name','bias'],
                         columns='year', values='n',
                         fill_value=0)
            .reset_index())
year_cols = [c for c in df_pivot.columns if c.isdigit()]
df_pivot[year_cols] = df_pivot[year_cols].astype(int)

In [24]:
df_pivot = df_pivot.sort_values('2021', ascending=False)
df_pivot.head(20)

year,outlet_name,bias,2015,2016,2017,2018,2019,2020,2021
19,Newsweek,Left,2180,2843,3964,4576,4018,5210,9547
4,Breitbart News,Right,1684,2396,2091,2495,3133,4493,6095
18,Newsmax,Unknown,1202,789,957,1078,395,1004,5288
17,New York Post,Unknown,633,717,847,2209,1777,3118,3855
7,CNBC,Center,880,801,1348,1532,2040,3229,3646
9,Daily Beast,Left,1936,2023,1623,2219,2680,3887,2706
42,Washington Times,Lean Right,1145,759,1672,1547,1345,1870,2525
29,The Epoch Times,Lean Right,128,177,27,82,638,166,2413
34,The Washington Post,Unknown,1295,1375,1426,1019,824,900,2310
40,Vox,Left,7824,4620,4332,3368,2083,3164,2010


In [25]:
# Filter to 3 outlets per political bias

OUTLETS = (
    "Newsweek",
    "The Washington Post",
    "HuffPost",
    "Breitbart News",
    "New York Post",
    "Washington Times",
    "CNBC",
    "Reuters",
    "BBC News"
)

query = f"""
SELECT 
    u.outlet_name,
    u.bias,
    c.date,
    c.content,
    c.content_preprocessed
FROM article_urls u
JOIN article_contents c
    ON u.uuid = c.uuid
WHERE c.date IS NOT NULL
  AND c.date >= '2015-01-01'
  AND c.date <  '2022-01-01'
  AND u.outlet_name IN ({','.join(['?']*len(OUTLETS))});
"""

LRC_articles = pd.read_sql(query, conn, params=list(OUTLETS))
LRC_articles.head()

,outlet_name,bias,date,content,content_preprocessed
0,HuffPost,Left,2017-03-13,The term “integrative therapies” describes the...,The term “integrative therapies” describes the...
1,HuffPost,Left,2018-02-28,"“I had people ― wealthy, billionaires ― callin...","“I had people ― wealthy, billionaires ― callin..."
2,HuffPost,Left,2018-09-24,"Le chef du Parti québécois, Jean-François Lisé...","Le chef du Parti québécois, Jean-François Lisé..."
3,HuffPost,Left,2019-03-06,James Leynse via Getty Images\nWhen is addicti...,James Leynse via Getty Images\nWhen is addicti...
4,HuffPost,Left,2015-02-10,"AMC\nFor Canadian actor Michael Mando, lightni...","AMC\nFor Canadian actor Michael Mando, lightni..."


In [30]:
LRC_articles.isna().sum()

outlet_name                 0
bias                    22305
date                        0
content                  4634
content_preprocessed        0
dtype: int64

In [31]:
LRC_articles[LRC_articles["bias"].isna()]["outlet_name"].unique()

array(['The Washington Post', 'New York Post'], dtype=object)

In [32]:
# Update political bias where missing due to naming convention (The Washington Post vs Washington Post)

bias_fix = {
    "The Washington Post": "Lean Left",
    "New York Post": "Lean Right",
}

LRC_articles["bias"] = LRC_articles.apply(
    lambda row: bias_fix.get(row["outlet_name"], row["bias"]),
    axis=1
)

LRC_articles.isna().sum()

In [39]:
# Drop records with no content)
LRC_articles = LRC_articles.dropna()

In [40]:
LRC_articles.isna().sum()

outlet_name             0
bias                    0
date                    0
content                 0
content_preprocessed    0
dtype: int64

In [41]:
LRC_articles.groupby("outlet_name").size()

outlet_name
BBC News               16205
Breitbart News         22361
CNBC                   13343
HuffPost               19373
New York Post          13154
Newsweek               32323
Reuters                11144
The Washington Post     9144
Washington Times       10697
dtype: int64

In [ ]:
LRC_articles.to_parquet("LRC_articles.parquet", index=False)